***PARTE DE PYTHON DE LA PRÁCTICA NÚMERO DOS DE DATOS***

(1) PREPARACIONES

In [1]:
import numpy as np

In [30]:
import pandas as pd

titanicTraindata = pd.read_csv("train.csv",quotechar='"') 


In [3]:
# Veamos la estructura de la tabla en Python

print (titanicTraindata.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None


In [4]:
# Encontramos el número de variables (columnas del DataFrame) que contienen NA

# La cantidad por columnas es:

titanicTraindata.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [5]:
titanicTraindata.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


(2) IMPUTACIONES DE VALORES PERDIDOS

In [6]:
from sklearn.impute import KNNImputer as KNNIM

# Creamos un DF auxuliar con los campos numéricos que nos permitan hacer una imputación por distancia euclidea
aux_df = titanicTraindata[['Age','Pclass','SibSp','Parch','Fare']]

imputer = KNNIM(n_neighbors=3)
aux_df2 = imputer.fit_transform(aux_df)

nueva_age = (    (aux_df2[:,0])  )

print (nueva_age.shape)

# Redondeamos las fechas a la parte entera de la edad.
nueva_age = nueva_age.astype(int)

print (np.unique(nueva_age))

(891,)
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 70 71 74 80]


In [7]:

titanicTraindata[['Age']] = (nueva_age)

# Para el valor Embartked asignamos el valor más comun = "S"

titanicTraindata[['Embarked']] = titanicTraindata[['Embarked']].fillna('S')


(3) DETERMINACIÓN OUTLIERS

In [8]:
# Encontramos los outliers

# Q3 +1.5⋅IQR
# Q1 −1.5⋅IQR

def limites_outliers (x):
    Q3 = np.percentile(x, 75)
    Q1 = np.percentile(x, 75)
    IQR = np.subtract(np.percentile(x, 75),np.percentile(x, 25))
    lim_inf = Q1 - 1.5 *IQR
    lim_sup = Q3 + 1.5 *IQR
    resultado = (lim_inf,lim_sup)               
    return resultado


In [9]:
# Encontramos los valores outliers del dataset para la variable "SibSp"

aux = titanicTraindata['SibSp']
aux2 = (limites_outliers (aux))
aux2 = titanicTraindata.loc[ (titanicTraindata['SibSp'] < aux2[0]) | (titanicTraindata['SibSp'] > aux2[1]) ] 

print (list(aux2.SibSp))

[3, 4, 3, 3, 4, 5, 3, 4, 5, 3, 3, 4, 8, 4, 4, 3, 8, 4, 8, 3, 4, 4, 4, 4, 8, 3, 3, 5, 3, 5, 3, 4, 4, 3, 3, 5, 4, 3, 4, 8, 4, 3, 4, 8, 4, 8]


In [10]:
# Encontramos los valores outliers del dataset para la variable "Parch"  
    
campo = 'Parch'
aux = titanicTraindata[campo]
aux2 = (limites_outliers (aux))
                      
aux2 = titanicTraindata.loc[ (titanicTraindata[campo] < aux2[0]) | (titanicTraindata[campo] > aux2[1]) ] 

print (list(aux2[campo]))

[1, 2, 1, 5, 1, 1, 5, 2, 2, 1, 1, 2, 2, 2, 1, 2, 2, 2, 3, 2, 2, 1, 1, 1, 1, 2, 1, 1, 2, 2, 1, 2, 2, 2, 1, 2, 1, 1, 2, 1, 4, 1, 1, 1, 1, 2, 2, 1, 2, 1, 1, 1, 2, 1, 1, 2, 2, 2, 1, 1, 2, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 2, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 4, 1, 1, 2, 2, 2, 2, 2, 1, 1, 1, 2, 2, 1, 1, 2, 2, 3, 4, 1, 2, 1, 1, 2, 1, 2, 1, 2, 1, 1, 2, 2, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 1, 1, 2, 1, 4, 1, 1, 2, 1, 2, 1, 1, 2, 5, 2, 1, 1, 1, 2, 1, 5, 2, 1, 1, 1, 2, 1, 6, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 3, 2, 1, 1, 1, 1, 2, 1, 2, 3, 1, 2, 1, 2, 2, 1, 1, 2, 1, 2, 1, 2, 1, 1, 1, 2, 1, 1, 2, 1, 2, 1, 1, 1, 1, 3, 2, 1, 1, 1, 1, 5, 2]


In [11]:
# Encontramos los valores outliers del dataset para la variable "Fare"  

campo = 'Fare'
aux = titanicTraindata[campo]
aux2 = (limites_outliers (aux))
aux2 = titanicTraindata.loc[ (titanicTraindata[campo] < aux2[0]) | (titanicTraindata[campo] > aux2[1]) ] 

print (list(aux2[campo]))

[71.2833, 263.0, 146.5208, 82.1708, 76.7292, 80.0, 83.475, 73.5, 263.0, 77.2875, 247.5208, 73.5, 77.2875, 79.2, 66.6, 69.55, 69.55, 146.5208, 69.55, 113.275, 76.2917, 90.0, 83.475, 90.0, 79.2, 86.5, 512.3292, 79.65, 153.4625, 135.6333, 77.9583, 78.85, 91.0792, 151.55, 247.5208, 151.55, 110.8833, 108.9, 83.1583, 262.375, 164.8667, 134.5, 69.55, 135.6333, 153.4625, 133.65, 66.6, 134.5, 263.0, 75.25, 69.3, 135.6333, 82.1708, 211.5, 227.525, 73.5, 120.0, 113.275, 90.0, 120.0, 263.0, 81.8583, 89.1042, 91.0792, 90.0, 78.2667, 151.55, 86.5, 108.9, 93.5, 221.7792, 106.425, 71.0, 106.425, 110.8833, 227.525, 79.65, 110.8833, 79.65, 79.2, 78.2667, 153.4625, 77.9583, 69.3, 76.7292, 73.5, 113.275, 133.65, 73.5, 512.3292, 76.7292, 211.3375, 110.8833, 227.525, 151.55, 227.525, 211.3375, 512.3292, 78.85, 262.375, 71.0, 86.5, 120.0, 77.9583, 211.3375, 79.2, 69.55, 120.0, 93.5, 80.0, 83.1583, 69.55, 89.1042, 164.8667, 69.55, 83.1583]


In [12]:
# Encontramos los valores outliers del dataset para la variable "Age"  

campo = 'Age'
aux = titanicTraindata[campo]
aux2 = (limites_outliers (aux))
aux2 = titanicTraindata.loc[ (titanicTraindata[campo] < aux2[0]) | (titanicTraindata[campo] > aux2[1]) ] 

print (list(aux2[campo]))

[2, 14, 4, 58, 14, 2, 8, 66, 14, 3, 4, 7, 65, 5, 11, 4, 0, 59, 71, 14, 70, 2, 12, 12, 9, 1, 9, 61, 4, 1, 9, 1, 4, 3, 58, 2, 59, 5, 8, 62, 3, 58, 63, 7, 65, 2, 0, 61, 2, 3, 60, 3, 1, 1, 3, 10, 14, 64, 4, 13, 5, 65, 0, 2, 9, 63, 58, 9, 71, 2, 7, 9, 11, 64, 8, 62, 62, 60, 4, 61, 80, 9, 2, 0, 58, 70, 14, 60, 14, 4, 60, 6, 11, 70, 4, 6, 0, 5, 13, 8, 1, 11, 0, 6, 10, 2, 1, 62, 0, 4, 74, 9, 4]


(4) TESTS DE NORMALIDAD NO PARAMÉTRICOS

In [13]:
' Test de normalidad mediante el test de Shapiro-Wilk'


from scipy.stats import shapiro


# Test de normalidad de la variable Pclass Resultado de (Prob,  "p" o singificancia)

x1 = shapiro(titanicTraindata.Pclass)

print (x1)

# Test de normalidad de la variable Age Resultado de (Prob,  "p" o singificancia)

x2 = shapiro(titanicTraindata.Age)
print (x2)

# Test de normalidad de la variable SibSp Resultado de (Prob,  "p" o singificancia)

x3 = shapiro(titanicTraindata.SibSp)
print (x3)

# Test de normalidad de la variable Parch Resultado de (Prob,  "p" o singificancia)

x4 = shapiro(titanicTraindata.Parch)
print (x4)

# Test de normalidad de la variable Fare Resultado de (Prob,  "p" o singificancia)

x5 = shapiro(titanicTraindata.Fare)
print (x5)


(0.718337893486023, 3.3958319924210316e-36)
(0.98072749376297, 1.8490181608044054e-09)
(0.5129655003547668, 5.74532370373175e-44)
(0.5328145027160645, 2.382207389352189e-43)
(0.5218914747238159, 1.0789998175301091e-43)


(5) TEST VARIANCIAS.

In [14]:
from scipy.stats import fligner as FK 

# En el meteodo encontrado en la libreria hemos podido comprobar que no se admiten campos no numéricos.
# Por ello los campos tipo STR  en 'Sex' , los convertimos en numéricos.

Sex_int = titanicTraindata.Sex

Sex_int = Sex_int.replace('female',0)
Sex_int = Sex_int.replace('male',1)

x1 = FK(Sex_int, titanicTraindata.Survived )

print (x1)

FlignerResult(statistic=1.8903305462894795, pvalue=0.16916464402951714)


In [15]:

# En el meteodo encontrado en la libreria hemos podido comprobar que no se admiten campos no numéricos.
# Por ello los campos tipo STR  en 'Embarked' , los convertimos en numéricos.

Embarked_int = titanicTraindata.Embarked

Embarked_int = Embarked_int.replace('C',0)
Embarked_int = Embarked_int.replace('Q',1)
Embarked_int = Embarked_int.replace('S',2)

x1 = FK(Embarked_int, titanicTraindata.Survived )

print (x1)

FlignerResult(statistic=4.124670954313547, pvalue=0.042262238657693296)


In [16]:
Pclass_int = titanicTraindata.Pclass

x1 = FK(Pclass_int, titanicTraindata.Survived )

print (x1)

FlignerResult(statistic=89.50879252308928, pvalue=3.0527868978165962e-21)


(6) CALCULOS DE CORRELACION DE SPEARMAN

In [17]:
# Aplicamos la correlación de Spearman.
# Hipótesis nula: No existe correlación significativa entre ambas variables.

from scipy.stats import spearmanr as SPR

x1 = SPR(titanicTraindata.Age , titanicTraindata.Survived )

print (x1)

SpearmanrResult(correlation=-0.03319443011904244, pvalue=0.3223095243860028)


In [18]:

x1 = SPR(titanicTraindata.SibSp , titanicTraindata.Survived )

print (x1)

SpearmanrResult(correlation=0.08887948468090501, pvalue=0.007941431285733533)


In [19]:

x1 = SPR(titanicTraindata.Parch , titanicTraindata.Survived )

print (x1)

SpearmanrResult(correlation=0.13826563286545587, pvalue=3.453591460380432e-05)


In [20]:

x1 = SPR(titanicTraindata.Fare , titanicTraindata.Survived )

print (x1)

SpearmanrResult(correlation=0.32373613944480834, pvalue=3.471227970207005e-23)


(7) TEST DE CHI PARA VARIABLE Sex

In [21]:
from scipy.stats import chi2_contingency as chi2


# Aplicamos un test de ChiSq.
# Hipótesis nula: No existe correlación significativa entre ambas variables Sex / Survived
# Para ello empleamos la lista numérica calculada anteriormente Sex_int
# Obtenemos la tabla de frecuencias

contingency_table = pd.crosstab( titanicTraindata.Survived, titanicTraindata.Sex )

print ( contingency_table )

x1 = chi2( contingency_table )

print ("--------------------")
print ("El valor del Estadístico de comprobación es")
print ( x1[0])
print ("--------------------")
print ("La 'p' calculada es:")
print ( x1[1])


Sex       female  male
Survived              
0             81   468
1            233   109
--------------------
El valor del Estadístico de comprobación es
260.71702016732104
--------------------
La 'p' calculada es:
1.1973570627755645e-58


(8) TEST DE WILCOX 

In [22]:
from scipy.stats import mannwhitneyu as mann

# Seleccionamos el grupo de pasajeros que no sobrevivieron 

titanicTraindata_died = titanicTraindata.loc[titanicTraindata['Survived'] == 0]
fare_died = titanicTraindata_died.Fare

titanicTraindata_surv = titanicTraindata.loc[titanicTraindata['Survived'] == 1] 
fare_surv = titanicTraindata_surv.Fare


stat, p = mann(x= fare_died, y= fare_surv, use_continuity= True, alternative="less")

print (stat, p)



57806.5 2.2767385896251186e-22


(9) DETERMINACION DE UNA FORMULA DE REGRESION LOGISTICA

In [23]:
y = titanicTraindata[['Survived']]
X = titanicTraindata[['Fare','Sex','Pclass','Age','SibSp','Embarked']]


# Tenemos que tratar la variables categoricas y las convertimos en numericas.

X = X.replace('female',0)
X = X.replace('male',1)

X = X.replace('C',0)
X = X.replace('Q',1)
X = X.replace('S',2)

In [25]:
import statsmodels.api as sm


# Realizamos una primera prueba por Statsmodels que nos da un resumen interesante del contenido del modelo.
logit2 = sm.Logit(y,X)
result2 = logit2.fit()
print (result2.summary2())


Optimization terminated successfully.
         Current function value: 0.495996
         Iterations 6
                         Results: Logit
Model:              Logit            Pseudo R-squared: 0.255     
Dependent Variable: Survived         AIC:              895.8653  
Date:               2020-06-04 13:29 BIC:              924.6193  
No. Observations:   891              Log-Likelihood:   -441.93   
Df Model:           5                LL-Null:          -593.33   
Df Residuals:       885              LLR p-value:      2.5168e-63
Converged:          1.0000           Scale:            1.0000    
No. Iterations:     6.0000                                       
------------------------------------------------------------------
              Coef.   Std.Err.     z      P>|z|    [0.025   0.975]
------------------------------------------------------------------
Fare          0.0172    0.0028    6.0364  0.0000   0.0116   0.0227
Sex          -2.2518    0.1751  -12.8604  0.0000  -2.5950  -1.

In [26]:
from sklearn.linear_model import LogisticRegression as LGR


# Realizamos el otro métido por Sklearn.
logit = LGR()
result=logit.fit(X,y)


/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [27]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

# Realizamos la predicción de probabilidades.
y_pred = result.predict(X)


confusion_matrix = confusion_matrix(y, y_pred)
print((confusion_matrix))


print(classification_report(y, y_pred))


[[470  79]
 [ 99 243]]
              precision    recall  f1-score   support

           0       0.83      0.86      0.84       549
           1       0.75      0.71      0.73       342

    accuracy                           0.80       891
   macro avg       0.79      0.78      0.79       891
weighted avg       0.80      0.80      0.80       891



In [28]:
import matplotlib.pyplot as plt 
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

logit_roc_auc = roc_auc_score(y, result.predict(X))
fpr, tpr, thresholds = roc_curve(y, result.predict_proba(X)[:,1])
plt.figure()
plt.plot(fpr, tpr, label='Logistic Regression (area = %0.2f)' % logit_roc_auc)
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.savefig('Log_ROC')
plt.show()


In [29]:
# GUARDAMOS EL CSV RESULTADO

titanicTraindata.to_csv('titanicTraindata_python.csv')  
